# Using inaFaceAnalyzer API: a video analysis quick-start tutorial
In this tutorial, we use inaFaceAnalyzer with default analysis parameters and export results to CSV, rich ASS subtitles and incrusted MP4. We also introduce FPS runtime argument allowing to speed-up analyses.

## Install inaFaceAnalyzer

In [ ]:
# try to import inaFaceAnalyzer and import it from Pypi's
# if it is not available
try:
  import inaFaceAnalyzer
except:
  # install inaFaceAnalyzer Pypi's distribution
  !pip install inaFaceAnalyzer

## Define and display sample video input

In [ ]:
# allow to display videos in jupyter and collab notebooks
from inaFaceAnalyzer.notebook_utils import notebook_display_remote_vid, notebook_display_local_vid

# input materials can be provided using file paths or remote urls
sample_vid = 'https://github.com/ina-foss/inaFaceAnalyzer/raw/master/media/pexels-artem-podrez-5725953.mp4'
# set desired width used for displaying video
notebook_display_remote_vid(sample_vid)

## Analyse a video with default parameters

In [ ]:
#import the video processing engine
from inaFaceAnalyzer.inaFaceAnalyzer import VideoAnalyzer

In [ ]:
# create a video analyzer instance
# Required classifications and detection models are first downloaded from remote location
# machine learning models loading is done during analyzer constructor and may require several seconds
# consequently, users should use the same analyzer instance to process several documents
#
# Analyzer's constructor may accept several parameters allowing
# to customize the processings to be done
# These parameters will be covered in more advanced tutorials
va = VideoAnalyzer()

In [ ]:
# Analyzers are designed as 'functions objects' or 'functors'
# see https://en.wikipedia.org/wiki/Function_object
# ie: analyzer instances can be used as functions, executing the code implemented in __call__ methods
# this example will have "long" processing time since all video frames will be processed
df = va(sample_vid)

In [ ]:
# Analysis results are returned as pandas DataFrames
# see https://pandas.pydata.org/docs/
# Results contain one line per detected faces and several columns :
#
# frame: the position of the frame in the video stream
# bbox: (left, top, right, bottom) the bounding box of the face in the image frame
# detect_conf: the face detection confidence estimate (dependent on the face detection method used)
# sex_decfunc: raw gender classifier output : positive values are used for men and negative values for women
# sex_label: gender classifer prediction: 'm' for men and 'w' for 'women'
# age_decfunc: raw age regression output based on FairFace age categories.
# 0 for (0-3 years old), 1 for (4-9) years, 2 for (10-19)  years, 3 for (20-29)  years, etc...
# sex_label : "human-readable" age prediction
#
# In this example, we found 1313 faces in a 12 second video sampled at 30 FPS
df

## Faster analyses with FPS argument
In the previous example, we performed face detection and classification for each video frames, using a video containing 30 frames per seconds.
While this strategy provide the most robust results, it is **SLOW** and not suited to large-scale processing.
`inaFaceAnalyzer` allow to define `fps` argument: the amount of frames to be processed of each second of video.
This allows users to define proper trade-offs between the amount of data to process and the available computational resources.

In [ ]:
# we define fps=2 in order to process only 2 frames for each second of video
# this is faster
ANALYSIS_FPS = 2
df_fps2 = va(sample_vid, fps=ANALYSIS_FPS)

In [ ]:
# when analysing only 2 frames per second of video, process only
# 24 images instead of 345, and we find 89 faces (instead of 1313)
# in the current 12 second except of video
df_fps2

## Exporting analysis results

In [ ]:
# dataframes are easy to export in CSV or to any tabular format
df.to_csv('./myexport.csv')
# display the resulting csv
!cat myexport.csv

## Exporting results to rich subtitles
Ass subtitles allow to display detected faces and classification results in VLC or ELAN
Subtitles are a good option for sharing results, since they do not require a large amount of storage size, and do not alter original videos

In [ ]:
from inaFaceAnalyzer.display_utils import ass_subtitle_export

In [ ]:
# export results to ass
# requires
# arg1 : input video filename or URL
# arg2 : analysis result (as a pandas dataframe OR as a resulting csvfile)
# arg3 : export ass subtitle file name
# arg4: analysis FPS not used with default FPS parameters
ass_subtitle_export(sample_vid, df, './mysample.ass')

In [ ]:
# download video: VLC cannot display subtitles on a remote video
!wget $sample_vid -O ./sample_vid.mp4
# open original video with the subtitle file in VLC (cannot be done in google collab)
!vlc --sub-file ./mysample.ass ./sample_vid.mp4

In [ ]:
# Get ass subtitles for results obtained with analysis FPS = 2
# When exporting results that were obtained with fps option,
# you should provide the value of analysis FPS that was used
ass_subtitle_export(sample_vid, df_fps2, './mysample_FPS2.ass', ANALYSIS_FPS)
!vlc --sub-file ./mysample_FPS2.ass ./sample_vid.mp4

## Exporting results to incrusted MP4 Videos
We provide result export options to MP4
MP4 export is slow and generate large files, we recommed using ASS subtitle export when possible

In [ ]:
from inaFaceAnalyzer.display_utils import video_export
from inaFaceAnalyzer.notebook_utils import notebook_display_local_vid

In [ ]:
# export results to MP4
# requires
# arg1 : input video filename or URL
# arg2 : analysis result (as a pandas dataframe OR as a resulting csvfile)
# arg3 : resulting incrusted MP4 file name
# arg4: analysis FPS not used with default analysis option
video_export(sample_vid, df, './myexportedvid.mp4')

In [ ]:
notebook_display_local_vid("./myexportedvid.mp4")

In [ ]:
# Export incrusted MP4 for results obtained with analysis FPS = 2
# Resulting MP4 will also contain 2 Frames per second
# When exporting results that were obtained with fps option,
# you should provide the value of analysis FPS that was used
video_export(sample_vid, df_fps2, './myexportedvid_fps2.mp4', ANALYSIS_FPS)

In [ ]:
notebook_display_local_vid("./myexportedvid_fps2.mp4")